## Ajuste de distribuciones


In [10]:
import neufipy as npy # solo la importo para usar los margenes
import pandas as pd #se necesita dentro de la funcion
import numpy as np # Se necesita dentro de la funcion
from scipy.optimize import curve_fit # es la funcion que ajusta
from scipy.stats import norm # Esto solo l uso para generar datos ficticios
import plotly.graph_objects as go

npy.use_margin()

In [69]:
def funcion(x,mu,sigma):
    ''' Esta es la funcion donde tenes que escribir de manera simbolica la funcion con la que queres ajustar. 
    El primer argumento tiene que ser "x" y despues todos los parametros que necesites '''
    return((np.exp(-0.5*((x-mu)/sigma)**2))/(sigma*((2*np.pi))**0.5)) # Puse una gaussiana aca porque estoy probando con una distribucion normal

In [70]:
def ajuste(func, data, bines = 50,rango = [0,0]):
    '''Esta funcion hace un ajuste de la PDF de los datos crudos que tiene en el argumento "data".
    func: es el nombre de la funcion en la que escribiste la funcion con la que vas a ajustar
    data: es una lista con los valores crudos. Por ejemplo todos los tiempos de todas las fijaciones
    bines: Con cuantos bines queres hacer el histograma a ajustar. Si no le pones ningun valor va a usar 50 bines.
    rango: es el rango del histograma, por defecto va a usar de rango desde el cuantil 5% al 95%
    La funcion devuelve:
    popt (parametros optimos): Los parametros del ajuste de la funcion. Es un array donde los parametros estan en el mismo orden que en la funcion definida
    pcov: Es la matriz de covarianza estimada.
    perr: Es la desviacion o error estandar de los parametros. Un array en el mismo orden que popt.
    datos_df: Es un DataFrame con la distribucion. Son los puntos medios de los bines y su altura normalizada'''
    if rango!=[0,0]:
        pass
    else:
        # Aca podes definir vos como queres que te tome el rango por defecto
        #rango = [np.quantile(data,0.005),np.quantile(data,0.995)]
        rango = [min(data),max(data)]
        
    bins_values,bins_edges = np.histogram(data,bins = bines,range = rango,density = True)
    delta = bins_edges[1]-bins_edges[0]
    # Transformo el resultado de la funcion np.histogram en un DataFrame.
    datos_df = pd.DataFrame({'bins':bins_edges[1:]-delta, 
                            'bins_values': bins_values})
    xdata = datos_df.bins
    ydata = datos_df.bins_values
    popt, pcov = curve_fit(func, xdata, ydata) # curve_fit es la funcion que hace el ajuste con la funcion "func" y los datos xdata e ydata
    perr = np.sqrt(np.diag(pcov)) # asi se calcula el error estadistico
    return(popt,pcov,perr,datos_df)


In [75]:
# Genero datos ransom que siga una distribucion normal
r = norm.rvs(size=10000,loc = 1,scale = 0.8) #ver https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html. 
# Utilizo la funcion de ajuste que definimos arriba
popt,pcov,perr,distri_df = ajuste(funcion,r) 
print(f'X promedio = [{popt[0]}  \u00B1 {perr[0]}] \nSigma = [{popt[1]} \u00B1 {perr[1]}]')#  print(f'texto{varible numerica}'') es para imprimir texto con variables numericas todo como texto

X promedio = [0.9275605717060406  ± 0.00720820419865805] 
Sigma = [0.8051122323954316 ± 0.005885738788262728]


### Grafico para la verificacion del ajuste

In [76]:
x_fit = np.linspace(min(distri_df.bins),max(distri_df.bins),len(distri_df)*5) # Dominio del ajuste
y_fit = funcion(x_fit,popt[0],popt[1]) # Imagen del ajuste
# Empiezo la grafica
fig = go.Figure()
fig.add_traces(go.Bar(x = distri_df.bins,y = distri_df.bins_values, name = "Distribucion"))
fig.add_traces(go.Scatter(x = x_fit,y = y_fit,name = 'Fit'))
# Voy a utilizar la funcion "estetica" del paquete neufipy para que la grafica tenga la estetica de paper
npy.estetica(fig,x_name = 'X Variable', y_name = 'Density Probability',title = 'Histograma con ajuste')
fig.show() # Cuando se utiliza npy.estetica hay que agregar fig.estetica.